# Preparation

In [1]:
from azureml.core import Workspace
import pandas as pd

In [2]:
ws = Workspace.get(name='windex', subscription_id='21868db0-87fa-4f03-bb0f-db7749ad7c9f', resource_group='Darwin-Dataanalysis-ML')

In [3]:
ws.get_details()

{'id': '/subscriptions/21868db0-87fa-4f03-bb0f-db7749ad7c9f/resourceGroups/Darwin-Dataanalysis-ML/providers/Microsoft.MachineLearningServices/workspaces/windex',
 'name': 'windex',
 'location': 'northeurope',
 'type': 'Microsoft.MachineLearningServices/workspaces',
 'tags': {},
 'workspaceid': '084f563d-7c9d-47f0-92bc-d00cca39b4f4',
 'description': '',
 'friendlyName': '',
 'creationTime': '2019-06-27T16:04:36.5228409+00:00',
 'containerRegistry': '/subscriptions/21868db0-87fa-4f03-bb0f-db7749ad7c9f/resourceGroups/Darwin-Dataanalysis-ML/providers/Microsoft.ContainerRegistry/registries/windex196f9b13',
 'keyVault': '/subscriptions/21868db0-87fa-4f03-bb0f-db7749ad7c9f/resourcegroups/darwin-dataanalysis-ml/providers/microsoft.keyvault/vaults/windex2307198450',
 'applicationInsights': '/subscriptions/21868db0-87fa-4f03-bb0f-db7749ad7c9f/resourcegroups/darwin-dataanalysis-ml/providers/microsoft.insights/components/windex1034289974',
 'identityPrincipalId': 'b675b9c2-d0e5-4672-b884-ac3c81c98

In [4]:
from azureml.core.datastore import Datastore
from azureml.data.data_reference import DataReference

ds = ws.get_default_datastore()
dataref = DataReference(datastore=ds)


In [6]:
ds_era5 = Datastore.register_azure_blob_container(workspace=ws, 
                                             datastore_name='era5', 
                                             container_name="era5",
                                             account_name="darwinlabstorage", 
                                             account_key="w9UWsOPrsAel9KgVe43WvJ6e6+0T2XrRHSlHO05au6ghAgQGi78frrkwAFcFrdVUzDvsMjq/wRB2JOPna5vpaA==",
                                             create_if_not_exists=False)

In [7]:
datarefera5 = DataReference(datastore=ds_era5)

#filePathInput1 = os.path.join(dataref, '2T_Brazil_East2007.nc')


In [9]:
print(dataref, datarefera5)

$AZUREML_DATAREFERENCE_workspaceblobstore $AZUREML_DATAREFERENCE_era5


In [5]:
ds.upload(src_dir='./ncdata',
          overwrite=True,
          show_progress=True)

Uploading ./ncdata\U100_fr1979.nc
Uploading ./ncdata\U100_fr1980.nc
Uploaded ./ncdata\U100_fr1980.nc, 1 files out of an estimated total of 2
Uploaded ./ncdata\U100_fr1979.nc, 2 files out of an estimated total of 2


$AZUREML_DATAREFERENCE_workspaceblobstore

# Local test

In [10]:
from netCDF4 import Dataset
import numpy as np

In [42]:
dataset_U100_79  = Dataset('./ncdata/U100_fr1979.nc', 'r', format = 'NETCDF')
dataset_U100_80  = Dataset('./ncdata/U100_fr1980.nc', 'r', format = 'NETCDF')

dataset_U100_79.variables['u100'][:]

masked_array(
  data=[[[ -2.44769495,  -1.81998153,  -0.98656175, ..., -19.55663776,
          -19.68906676, -19.67758958],
         [ -2.93856508,  -2.42827203,  -1.67342678, ..., -18.41509986,
          -18.63140054, -18.70909222],
         [ -3.41795803,  -3.42678663,  -3.25021464, ..., -16.5937598 ,
          -16.8118262 , -17.05196411],
         ...,
         [  6.232584  ,   5.82735128,   5.40181279, ...,  15.59619654,
           14.30457245,  10.97707333],
         [  4.16492601,   3.80560202,   4.15521455, ...,  14.84576559,
           14.40521848,  12.10360261],
         [  6.68725687,   7.03863513,   7.40767058, ...,  14.38579556,
           14.51910741,  13.62123885]],

        [[ -2.04422796,  -1.6928497 ,  -1.00510181, ..., -19.13286499,
          -19.38006578, -19.54427772],
         [ -2.65958134,  -2.37353472,  -1.74405558, ..., -18.29238233,
          -18.46365716, -18.62345481],
         [ -3.02067105,  -3.13720856,  -2.95710514, ..., -16.39599917,
          -16.55491

In [44]:
dataset_U100_79  = Dataset('./ncdata/U100_fr1979.nc', 'r', format = 'NETCDF')
dataset_U100_80  = Dataset('./ncdata/U100_fr1980.nc', 'r', format = 'NETCDF')
U100_79 = dataset_U100_79.variables['u100'][:]
U100_80 = dataset_U100_80.variables['u100'][:]
df_output = pd.DataFrame(np.concatenate((U100_79, U100_80))[0])
print(df_output.head(10))

         0         1         2         3         4         5         6   \
0 -2.447695 -1.819982 -0.986562 -0.048964  0.584046  1.093456  1.180859   
1 -2.938565 -2.428272 -1.673427 -1.293797 -0.990093 -0.564555 -0.159322   
2 -3.417958 -3.426787 -3.250215 -2.903251 -2.587187 -2.144874 -0.960076   
3 -4.617765 -4.724591 -4.677799 -4.275215 -3.764922 -3.157514 -2.091020   
4 -5.769014 -5.859066 -6.011801 -5.473256 -4.842894 -4.107472 -3.417958   
5 -6.761349 -6.650991 -6.831977 -6.324333 -5.891732 -5.390267 -4.696339   
6 -7.443799 -7.148924 -6.701314 -6.195435 -5.792851 -5.475022 -4.970026   
7 -7.144510 -6.345522 -5.457364 -4.689276 -3.969745 -3.614836 -3.123083   
8 -7.504717 -6.782537 -5.939406 -4.703402 -3.477110 -2.655167 -1.955059   
9 -7.520608 -6.823149 -6.055944 -4.675151 -3.422372 -2.537747 -1.663715   

         7         8         9   ...         71         72         73  \
0  1.156139  0.742078  0.426897  ... -15.040809 -15.858337 -16.622011   
1  0.637901  0.866561  0.651

# server treatement

In [45]:
%%writefile ./scripts/transform_windex.py

import argparse
import os
import numpy as np
import pandas as pd

from netCDF4 import Dataset

# let user feed in 2 parameters, the location of the data files (from datastore), and the regularization rate of the logistic regression model
parser = argparse.ArgumentParser()
parser.add_argument('--dataFolderInput', type=str)
parser.add_argument('--dataFolderOutput', type=str)
args = parser.parse_args()

dataFolderInput = args.dataFolderInput
dataFolderOutput = args.dataFolderOutput

print('dataFolderInput =>', dataFolderInput, 'dataFolderOutput =>', dataFolderOutput)

filePathInput1 = os.path.join(dataFolderInput, 'france','U100_fr1979.nc')
filePathInput2 = os.path.join(dataFolderInput, 'france','U100_fr1980.nc')

dataset_U100_79  = Dataset(filePathInput1, 'r', format = 'NETCDF')
dataset_U100_80  = Dataset(filePathInput2, 'r', format = 'NETCDF')
U100_79 = dataset_U100_79.variables['u100'][:]
U100_80 = dataset_U100_80.variables['u100'][:]
df_output = pd.DataFrame(np.concatenate((U100_79, U100_80))[0])

filePathOutput = os.path.join(dataFolderOutput,'2Tbr.csv')
df_output.to_csv(filePathOutput)


Overwriting ./scripts/transform_windex.py


In [17]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

aml_compute_target = "cpu-cluster"
try:
    aml_compute = AmlCompute(ws, aml_compute_target)
    print("found existing compute target.")
except ComputeTargetException:
    print("creating new compute target")
    
    provisioning_config = AmlCompute.provisioning_configuration(vm_size = "STANDARD_D2_V2",
                                                                min_nodes = 1, 
                                                                max_nodes = 4)    
    aml_compute = ComputeTarget.create(ws, aml_compute_target, provisioning_config)
    aml_compute.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)
    
print("Azure Machine Learning Compute attached")

found existing compute target.
Azure Machine Learning Compute attached


In [18]:
from azureml.core.runconfig import RunConfiguration
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.runconfig import DEFAULT_CPU_IMAGE

run_config = RunConfiguration()
run_config.environment.docker.enabled = True
run_config.environment.docker.base_image = DEFAULT_CPU_IMAGE
run_config.environment.python.user_managed_dependencies = False

# specify CondaDependencies obj
run_config.environment.python.conda_dependencies = CondaDependencies.create(conda_packages=['scikit-learn', 'pandas'],pip_packages=['netCDF4', 'pyarrow'] )

In [46]:
from azureml.pipeline.core import Pipeline
from azureml.pipeline.steps import PythonScriptStep
from azureml.core import Experiment

step_read_transf = PythonScriptStep(name="transform_nc_step",
                         script_name="transform_windex.py",
                         arguments=['--dataFolderInput',datarefera5,'--dataFolderOutput',dataref],
                         inputs=[datarefera5, dataref],
                         compute_target=aml_compute, 
                         source_directory='./scripts',
                         runconfig=run_config)

In [47]:
pipeline_transform_windex = Pipeline(workspace=ws, steps=[step_read_transf])
pipeline_transform_windex_run = Experiment(ws, 'run_nc_pipe').submit(pipeline_transform_windex, regenerate_outputs=True)
pipeline_transform_windex_run

Created step transform_nc_step [dc654d03][0bf2ac77-4521-4eb9-8212-561ca5dbabc7], (This step will run and generate new outputs)
Created data reference era5 for StepId [a1957223][bcc3c126-cedd-47e0-b467-bd4c46ea08f6], (Consumers of this data will generate new runs.)
Created data reference workspaceblobstore for StepId [c7877bc0][596d8cf6-cc66-4d5a-b3b5-4f335c5ad684], (Consumers of this data will generate new runs.)
Submitted pipeline run: 26b7870e-398d-48f0-90ce-e7b3e2016597


Experiment,Id,Type,Status,Details Page,Docs Page
run_nc_pipe,26b7870e-398d-48f0-90ce-e7b3e2016597,azureml.PipelineRun,NotStarted,Link to Azure Portal,Link to Documentation


##  <span style="color:blue"> Prepare Class </span> TODO

In [1]:
from sklearn.base import BaseEstimator, TransformerMixin
import numpy as np

In [2]:
from netCDF4 import Dataset
import os
import pandas as pd
import numpy as np
import ask.connect
import matplotlib.pyplot as plt
import datetime
import calendar
from Ask4meteo import *

In [3]:
class WinProdex(BaseEstimator,TransformerMixin):
    
    def __init__(self, **params):
        
        self.params = params
        self.alpha = None
        self.csv20ans = None
    
    def fit(self, X, y=None, write_ref=True):

        Ws_10 = X['Ws_10'].values
        Ws_100 = X['Ws_100'].values
    
        self.alpha = self.turbine_height_power_law_extrapolation_fct(Ws_10, Ws_100, **self.params)
        
        if write_ref:
            self.csv20ans = self.transform(X)
        
        return self
    
    
    def transform(self, X, y=None):
        
        Ws_10 = X['Ws_10'].values
        res = Ws_10 * (np.log( self.params['z2']/self.params['z0']))/(np.log(self.params['z1']/self.params['z0']))
        return res
    

    def score_kpi(self, X):
        res = self.csv20ans.mean() - self.transform(X).mean()
        return res
    
    @staticmethod
    def turbine_height_power_law_extrapolation_fct(Ws_z1, Ws_z2, z0, z1, z2, z3, law):
        print(Ws_z1, Ws_z2, z0, z1, z2, z3, law)
        
        if law == '7th':
            return 1/7
        elif law == 'Hellman':
            return (np.log(np.mean(Ws_z2)) - np.log(np.mean(Ws_z1)))/(np.log(np.mean(z2)) - np.log(np.mean(z1)))
        elif law == 'JustusMikhail':
            return (0.37 - 0.0881*np.log(Ws_z1))/(1-0.0881*np.log(z1))
        elif law == 'Davenport': #---Surface rugosity
            return 
        elif law == 'Reed': # Wind speed at ref height
            return 
        elif law == 'Smith': #--- Atmospheric stratification
            return 
        elif law == 'Sisterson': #--- Atmospheric stratification
            return
        
    @staticmethod
    def XlonYlat_WT_fct(WT, db):
        """
        Get lat and lon of the Wind turbine
        """
        request = """
        SELECT distinct [MAC_LATITUDE],[MAC_LONGITUDE]
        FROM [dbo].CATALOGUE_MACHINE
        WHERE ("MAC_CODE" like '%{}%')
        """.format(WT)
        LonLat = pd.read_sql(request, db.engine)
        xlat, ylon =  float(LonLat['MAC_LATITUDE']), float(LonLat['MAC_LONGITUDE'])
        return xlat, ylon 
    
    @staticmethod
    def ReadEraWind_fct(CountryFile, Area, Date_Start, Date_End, Dir_ERA):
        DeltaT = 1#hour
        VarName = ['U10','V10','U100','V100']
        print(VarName[0],Date_Start, Date_End, CountryFile, DeltaT)
        U10, ListDate_wish,lat,lon = ReadNetCDFbyVar(VarName[0],Date_Start, Date_End, CountryFile, Area,Dir_ERA)
        V10 ,_,_,_ = ReadNetCDFbyVar(VarName[1],Date_Start, Date_End, CountryFile, Area,Dir_ERA)
        U100,_,_,_ = ReadNetCDFbyVar(VarName[2],Date_Start, Date_End, CountryFile, Area,Dir_ERA)
        V100,_,_,_ = ReadNetCDFbyVar(VarName[3],Date_Start, Date_End, CountryFile, Area,Dir_ERA)
        Ws_10 = np.sqrt( U10*U10+V10*V10)
        Ws_100 = np.sqrt(U100*U100+V100*V100)
        print("----------------->>>> Done")
        return Ws_10, Ws_100, lat, lon
    
 

In [35]:
db = ask.connect.connect_to_darwin()
CountryFile = 'fr'
WT = 'FRMVF_E1_1381'
Dir_ERA = 'X:\\MeteoData\\ERA5\\'


xlat, ylon = WinProdex.XlonYlat_WT_fct(WT,db)
Area  = [xlat, ylon]

#Fetch data 20 ans pour fitter et 1 an pour transformer
Ws_10_1an, Ws_100_1an, lat, lon = WinProdex.ReadEraWind_fct(CountryFile,Area,'2017-01-01', '2018-01-01', Dir_ERA)
Ws_10_20ans, Ws_100_20ans, lat, lon = WinProdex.ReadEraWind_fct(CountryFile,Area,'1998-01-01', '2018-01-01', Dir_ERA)

X_1an = pd.DataFrame({'Ws_10':Ws_10_1an, 'Ws_100':Ws_100_1an})
X_20ans = pd.DataFrame({'Ws_10':Ws_10_20ans, 'Ws_100':Ws_100_20ans})

U10 2017-01-01 2018-01-01 fr 1
-----------------------  U10  -----------------------
Nearest grid point Lat : 43.00   Lon : 3.00  48 32
Year :  |  2017 |  2018
--------------------------------------------------
Date start :  2017-01-01 00:00:00   |  Date End : 2018-01-01 23:00:00
-----------------------  V10  -----------------------
Nearest grid point Lat : 43.00   Lon : 3.00  48 32
Year :  |  2017 |  2018
--------------------------------------------------
Date start :  2017-01-01 00:00:00   |  Date End : 2018-01-01 23:00:00
-----------------------  U100  -----------------------
Nearest grid point Lat : 43.00   Lon : 3.00  48 32
Year :  |  2017 |  2018
--------------------------------------------------
Date start :  2017-01-01 00:00:00   |  Date End : 2018-01-01 23:00:00
-----------------------  V100  -----------------------
Nearest grid point Lat : 43.00   Lon : 3.00  48 32
Year :  |  2017 |  2018
--------------------------------------------------
Date start :  2017-01-01 00:00:00   |

In [40]:
uc = WinProdex(z0=0.15, z1=10, z2=100, z3 = 80, law='Hellman')
uc.fit(X_20ans,write_ref=True)

transf_20ans = uc.transform(X_20ans)
transf_1an = uc.transform(X_1an)
uc.score_kpi(X_1an)

[3.83817101 3.99248125 4.35130947 ... 6.90750838 6.7712165  5.0318868 ] [ 6.93168417  7.15464431  7.6684532  ... 11.92066421 11.77902268
  9.93553055] 0.15 10 100 80 Hellman


-0.08705081922719948